In [1]:
# Testing the downloading data method from:
# https://claut.gitlab.io/man_ccia/lab2.html
# Import the required modules
from pyesgf.search import SearchConnection
import os
import pandas as pd
import requests
from tqdm import tqdm

# Set the os environment to on
os.environ['ESGF_PYCLIENT_NO_FACETS_STAR_WARNING'] = "on"

In [ ]:
# Import the functions
from testing_download_functions import query_data_esgf

In [2]:
# set the search connection
# to the LLNL search node
conn = SearchConnection('https://esgf-node.llnl.gov/esg-search', distrib=True)

In [3]:
print(type(conn))

<class 'pyesgf.search.connection.SearchConnection'>


In [ ]:
# test the function for querying the database
results = query_data_esgf(conn,
                        source_id='CanESM5',
                        experiment_id='historical',)